# Dataset Diabetes

## Importações

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import math
import statistics

In [ ]:
# corrigir argumentos do método predict
# salvar referências dos registros em vez de recalcular uma nova base de dados a cada nó
# reduzir quantidade de laços de repetição
# avaliar necessidade de criação de classes para cada feature

## Leitura de dados

In [2]:
dados = pd.read_csv('diabetes_train.csv', sep=',')

In [3]:
teste = pd.read_csv('diabetes_test.csv', sep=',')

In [4]:
labels_features = list(dados.columns[0:len(dados.columns) - 1])

In [5]:
label_target = dados.columns[len(dados.columns) - 1]

In [6]:
x_teste = []

for i in range(0, len(teste)):
    buffer = {}
    for j in labels_features:
        valor = str(teste[j][i])
        if valor.isdigit():
            valor = int(valor)
        buffer[j] = valor
    x_teste.append(buffer)

y_teste = list(teste[label_target])

In [7]:
x_treinamento = pd.DataFrame(dados.loc[:][[i for i in labels_features]])
y_treinamento = pd.DataFrame(dados.loc[:, label_target])

## Decision Tree

In [8]:
class No:
    def __init__(self, tipo, classe = None, valor = None):
        self.tipo = tipo
        self.classe = classe
        self.valor = valor
        self.conexoes = {}
    
    def inicializar_conexoes(self):
        for i in self.classe.categorias:
            self.conexoes[i] = None


In [ ]:
class Classe:
    def __init__(self, nome, categorias, valores, tipo, prob_priori = None, prob_condicional = None, entropia = None, ganho = 0):
        self.nome = nome
        self.categorias = categorias
        self.valores = valores
        self.tipo = tipo,
        self.prob_priori = prob_priori
        self.prob_condicional = prob_condicional
        if tipo != 'target': self.entropia = {}
        else: self.entropia = entropia
        self.ganho = ganho
    
    def calcula_prob_priori(self):
        buffer = dict(Counter(self.valores))
        soma = sum(buffer.values())
        for i in buffer:
            buffer[i] = buffer[i]/soma
        self.prob_priori = buffer
    
    def calcula_prob_condicional(self, categorias_target, valores_target):
        buffer = {}

        for i in self.categorias:
            buffer[i] = {}

        for i in buffer:
            for j in categorias_target:
                buffer[i][j] = 0

        for i, j in zip(self.valores, valores_target):
            buffer[i][j] += 1

        for i in buffer:
            soma = sum(buffer[i].values())
            for j in buffer[i]:
                buffer[i][j] = buffer[i][j]/soma

        self.prob_condicional = buffer
        

In [ ]:
class ArvoreDecisao:
    def __init__(self):
        self.raiz = None

    def set_raiz(self, raiz):
        self.raiz = raiz

    def calcula_entropia(self, s):
        entropia = 0
        for i in s:
            if i != 0:
                entropia -= i * math.log(i, 2)
        
        return entropia

    def calcula_ganho(self, classe, target):
        for i in classe.categorias:
            classe.ganho += classe.entropia[i] * classe.prob_priori[i]
        
        classe.ganho = target.entropia - classe.ganho

    def calcula_maior_ganho(self, classes):
        ganhos = [classes[i].ganho for i in classes]
        
        for i in classes:
            if classes[i].ganho == max(ganhos):
                return classes[i]

    def calcula_nova_base_dados(self, no, conexao, target, features, labels_features):
        x_treinamento, y_treinamento = {}, {}

        for j in labels_features:
            if j is not no.nome:
                x_treinamento[j] = []

                for k, l in zip(no.valores, features[j].valores):
                    if k == conexao:
                        x_treinamento[j].append(l)

        y_treinamento[target.nome] = []

        for k, l in zip(no.valores, target.valores):
            if k == conexao:
                y_treinamento[target.nome].append(l)
        
        x_treinamento = pd.DataFrame(x_treinamento)
        y_treinamento = pd.DataFrame(y_treinamento)
        
        return x_treinamento, y_treinamento

    def calcula_arvore_decisao(self, x_treinamento, y_treinamento):
        labels_features = list(x_treinamento.columns[0:len(x_treinamento.columns)])
        label_target = y_treinamento.columns[0]
        
        x_treinamento = [list(x_treinamento.iloc[:][i].values) for i in labels_features]
        y_treinamento = list(y_treinamento.iloc[:, 0].values)

        target = Classe(label_target, list(Counter(y_treinamento).keys()), y_treinamento, 'target')
        target.calcula_prob_priori()
        target.entropia = self.calcula_entropia(list(target.prob_priori.values()))

        features = {}

        for i, j in enumerate(labels_features):
            features[j] = Classe(j, list(Counter(x_treinamento[i]).keys()), x_treinamento[i], 'feature')
            features[j].calcula_prob_condicional(list(Counter(y_treinamento).keys()), y_treinamento)
            features[j].calcula_prob_priori()

            for i in features[j].categorias:
                features[j].entropia[i] = self.calcula_entropia(list(features[j].prob_condicional[i].values()))

            self.calcula_ganho(features[j], target)

        classe = self.calcula_maior_ganho(features)

        if classe.ganho != 0:
            no = No('no', classe)
            no.inicializar_conexoes()
            for i in no.conexoes:
                x_treinamento, y_treinamento = self.calcula_nova_base_dados(classe, i, target, features, labels_features)
                no.conexoes[i] = self.calcula_arvore_decisao(x_treinamento, y_treinamento)
            return no
        elif len(list(Counter(y_treinamento).keys())) == 1:
            return No('folha', None, list(Counter(y_treinamento).keys())[0])
        
        return No('folha')
    
    def calcula_folha_maior_frequencia(self, no):
        lista = []
        for i in no.conexoes:
                j = no.conexoes[i]
                if(j.tipo == 'folha' and j.valor != None):
                    lista.append(j.valor)

        
        return statistics.mode(lista)
    
    def verifica_folhas_none(self, no):
        lista = []
        for i in no.conexoes:
                j = no.conexoes[i]
                if(j.tipo == 'folha'):
                    if(j.valor == None):
                        j.valor = self.calcula_folha_maior_frequencia(no)
                else:
                    lista.append(j)
        
        for i in lista:
            self.verifica_folhas_none(i)
        
    def exibir_arvore_decisao(self, no):
        lista = []
        print("==========================")
        print(f"Nó atual: {no.classe.nome}")
        for i in no.conexoes:
                j = no.conexoes[i]
                print(f"Categoria: {i} -> {j.tipo}")
                if(j.tipo != 'folha'):
                    print(f"\t -> {j.classe.nome}")
                    lista.append(j)
                else:
                    print(f"\tResultado: {j.valor}")
        print("==========================\n\n")
        for i in lista:
            self.exibir_arvore_decisao(i)
    
    def fit(self, x_treinamento, y_treinamento):
        if not isinstance(x_treinamento, pd.DataFrame) or not isinstance(y_treinamento, pd.DataFrame):
            print("Atenção: os dados de treinamento precisam ser dataframes!\n")
            return
        
        self.set_raiz(self.calcula_arvore_decisao(x_treinamento, y_treinamento))
        self.verifica_folhas_none(self.raiz)

    def arvore_decisao(self, instancia, no):
        if no.tipo != 'folha':
            for i in no.conexoes:
                if i == instancia[no.classe.nome]:
                    resultado = self.arvore_decisao(instancia, no.conexoes[i])
        else: 
            return int(no.valor)
        
        return resultado

    def predict(self, x_teste):
        resultados = []
        for i in x_teste:
            resultados.append(self.arvore_decisao(i, self.raiz))
        return resultados


In [11]:
arvore = ArvoreDecisao()
arvore.fit(x_treinamento, y_treinamento)
arvore.exibir_arvore_decisao(arvore.raiz)
resultados = arvore.predict(x_teste)

Nó atual: polyuria
Categoria: 1 -> no
	 -> polydipsia
Categoria: 0 -> no
	 -> polydipsia


Nó atual: polydipsia
Categoria: 1 -> folha
	Resultado: 1
Categoria: 0 -> no
	 -> delayed_healing


Nó atual: delayed_healing
Categoria: 0 -> folha
	Resultado: 1
Categoria: 1 -> no
	 -> obesity


Nó atual: obesity
Categoria: 0 -> no
	 -> weakness
Categoria: 1 -> folha
	Resultado: 0


Nó atual: weakness
Categoria: 1 -> folha
	Resultado: 1
Categoria: 0 -> no
	 -> age


Nó atual: age
Categoria: 60<_<=80 -> folha
	Resultado: 0
Categoria: 40<_<=60 -> folha
	Resultado: 1
Categoria: 20<_<=40 -> folha
	Resultado: 1


Nó atual: polydipsia
Categoria: 0 -> no
	 -> gender
Categoria: 1 -> no
	 -> muscle_stiffness


Nó atual: gender
Categoria: Male -> no
	 -> irritability
Categoria: Female -> no
	 -> alopecia


Nó atual: irritability
Categoria: 0 -> no
	 -> delayed_healing
Categoria: 1 -> no
	 -> genital_thrush


Nó atual: delayed_healing
Categoria: 0 -> folha
	Resultado: 0
Categoria: 1 -> no
	 -> age


Nó atua

## Métricas

In [12]:
acuracia = 0
for i, k in zip(y_teste, resultados):
    if i == k:
        acuracia += 1
acuracia = (acuracia/len(y_teste)) * 100

In [13]:
print(f"Acurácia: {acuracia:.2f}%")

Acurácia: 94.23%


In [14]:
matriz_confusao = np.zeros((2, 2))

for i, k in zip(y_teste, resultados):
    matriz_confusao[i][k] += 1

In [15]:
matriz_confusao

array([[ 79.,   2.],
       [ 10., 117.]])